In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Загрузка данных: пропускаем первую строку с заголовками
data = pd.read_csv('/home/djden/Science/KAN/datasets_raw/db_nl_preprocessed-edit.csv', skiprows=1)
X = data.iloc[:, :-1].values  # Признаки
y = data.iloc[:, -1].values   # Целевая переменная

# Разделение данных
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Стандартизация данных
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Преобразование в тензоры
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).unsqueeze(1)

In [6]:
# Импорт модели KAN из официальной реализации pykan
# Если импорт не проходит, попробуйте изменить путь, например:
# from pykan.model import KAN
from kan import KAN

dims = [X_train.shape[1], 64, 32, 1]
model = KAN(dims)

# Определение функции потерь и оптимизатора
criterion = nn.MSELoss()  # Для задачи регрессии; для классификации используйте другую функцию потерь
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Обучение модели
epochs = 100
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    loss.backward()
    optimizer.step()
    
    if (epoch + 1) % 10 == 0:
        print(f'Epoch {epoch+1}/{epochs}, Loss: {loss.item():.4f}')

# Оценка на тестовой выборке
model.eval()
with torch.no_grad():
    predictions = model(X_test_tensor)
    test_loss = criterion(predictions, y_test_tensor)
    print(f'Test Loss: {test_loss.item():.4f}')


checkpoint directory created: ./model
saving model version 0.0
Epoch 10/100, Loss: nan
Epoch 20/100, Loss: nan
Epoch 30/100, Loss: nan
Epoch 40/100, Loss: nan
Epoch 50/100, Loss: nan
Epoch 60/100, Loss: nan
Epoch 70/100, Loss: nan
Epoch 80/100, Loss: nan
Epoch 90/100, Loss: nan
Epoch 100/100, Loss: nan
Test Loss: nan
